# Lab 6: Polarization

## Part 1: Setup

In [ ]:
from __future__ import annotations

%pip install uncertainties

import gspread
import matplotlib.pyplot as plt
import numpy as np
import numpy.typing as npt
import pandas as pd
import scipy.optimize as opt
from google.auth import default
from google.colab import auth
from IPython.display import Markdown
from matplotlib.axes import Axes
from matplotlib.figure import Figure
from uncertainties import UFloat, ufloat

Make a copy [the template spreadsheet](https://docs.google.com/spreadsheets/d/1-4fm3c7IGjaL4IaAx2qJvQjsumT36zrd27G5B0AXcbA/edit?usp=sharing) in Google Drive.
Replace `NAME OF GOOGLE SHEETS FILE` in the next cell with the name of your copy of the spreadsheet.

In [ ]:
auth.authenticate_user()

creds, _ = default()

gc = gspread.authorize(creds)

gs = gc.open("NAME OF GOOGLE SHEETS FILE")

You will make use of the light source, three polarizers, a waveplate, and a sensor.
Start by placing the light source and the sensor on the rail and reading the sensor value when the light source is turned off.

Turn on the light source and observe the sensor reeding as you vary the distance between the source and sensor.
Find and record the distance at which the measurement saturates and the measured value at which the sensor saturates.

Now adjust the distance between the light source and sensor so that the sensor is just below saturation.
You will keep the light source and sensor at this separation for the remainder of the experiment.
Record the positions of both the light source and the sensor for reference.

In [ ]:
def cos_sq_fit(
    angle: npt.NDArray[np.floating] | pd.Series,
    height: float,
    angular_offset: float,
    background_offset: float,
) -> npt.NDArray[np.floating]:
    phase = np.asarray(angle) + angular_offset
    return height * np.cos(phase) ** 2 + background_offset


def cos_sq_sin_sq_fit(
    angle: npt.NDArray[np.floating] | pd.Series,
    height: float,
    angular_offset: float,
    background_offset: float,
) -> npt.NDArray[np.floating]:
    phase = np.asarray(angle) + angular_offset
    return height * np.cos(phase) ** 2 * np.sin(phase) ** 2 + background_offset

In [ ]:
ws: dict[int, gspread.Worksheet] = {}
df: dict[int, pd.DataFrame] = {}
intensity: dict[int, dict[str, UFloat | float]] = {}
popt: dict[int, npt.NDArray[np.floating]] = {}
pcov: dict[int, npt.NDArray[np.floating]] = {}
perr: dict[int, npt.NDArray[np.floating]] = {}
height: dict[int, UFloat] = {}
angular_offset: dict[int, UFloat] = {}
background_offset: dict[int, UFloat] = {}
fig: dict[int, Figure] = {}
axs: dict[int, dict[str, Axes]] = {}
x: dict[int, npt.NDArray[np.floating]] = {}

## Part 2: Single Polarizer

Now insert a single polarizer between the source and sensor.
Vary the angle of the polarizer from 0&deg; to 350&deg; in 10&deg; increments and record the sensor reading as a function of this angle.

Note the maximum and minimum values and the angles at which they oocure.

Is this what you expected?

What does this tell you about the polarization of the light source?

In [ ]:
part: int = 2
ws[part] = gs.worksheet(f"Part {part}: Data")
df[part] = pd.DataFrame(
    data=np.asarray(
        ws[part].batch_get(
            ["A2:B37"],
            value_render_option=gspread.utils.ValueRenderOption.unformatted,
        )[0]
    ),
    columns=[
        "angle",
        "intensity",
    ],
)

intensity[part] = {
    "nominal": ufloat(
        df[part]["intensity"].mean(), df[part]["intensity"].std()
    ),
    "max": df[part]["intensity"].max(),
    "min": df[part]["intensity"].min(),
}
display(
    Markdown(
        Rf"Nominal Intensity: ${intensity[part]['nominal']:SL}\,\text{{lx}}$"
        "\n\n"
        Rf"Maximum Intensity: ${intensity[part]['max']:.1f}\,\text{{lx}}$"
        "\n\n"
        Rf"Minimum Intensity: ${intensity[part]['min']:.1f}\,\text{{lx}}$"
    )
)

popt[part], pcov[part] = opt.curve_fit(
    cos_sq_fit,
    np.deg2rad(df[part]["angle"]),
    df[part]["intensity"],
    p0=[
        df[part]["intensity"].max() - df[part]["intensity"].min(),
        0,
        df[part]["intensity"].min(),
    ],
)
perr[part] = np.sqrt(np.diag(pcov[part]))
height[part] = ufloat(popt[part][0], perr[part][0])
angular_offset[part] = ufloat(
    np.rad2deg(popt[part][1]), np.rad2deg(perr[part][1])
)
background_offset[part] = ufloat(popt[part][2], perr[part][2])

df[part]["residuals"] = df[part]["intensity"] - cos_sq_fit(
    np.deg2rad(df[part]["angle"]), *popt[part]
)

display(
    Markdown(
        R"Fit: $I\left(\theta\right) = "
        R"I_0 \cos^2\left(\theta + \theta_0\right) + I_\text{bg}$"
        "\n"
        Rf"- Height: $I_0 = {height[part]:SL}\,\text{{lx}}$"
        "\n"
        Rf"- Angular Offset: $\theta_0 = {angular_offset[part]:SL}\degree$"
        "\n"
        R"- Background Offset: $I_\text{{bg}} = "
        Rf"{background_offset[part]:SL}\,\text{{lx}}$"
        "\n\n"
        Rf"$I_0 / I_\text{{bg}} = {height[part] / background_offset[part]:SL}$"
    )
)

fig[part], axs[part] = plt.subplot_mosaic(
    [["main"], ["res"]],
    sharex=True,
    height_ratios=[1, 0.5],
)
axs[part]["main"].set_title(R"Transmitted Intensity vs. Polarizer Angle")
axs[part]["res"].set_xlabel(R"Polarizer Angle $\left(\degree\right)$")
axs[part]["main"].set_ylabel(R"Transmitted Intensity $\left(\text{lx}\right)$")
axs[part]["res"].set_ylabel(R"Residuals $\left(\text{lx}\right)$")
axs[part]["main"].scatter(
    df[part]["angle"],
    df[part]["intensity"],
    label="Measured Intensity",
)
x[part] = np.linspace(
    min(0, df[part]["angle"].min()),
    max(360, df[part]["angle"].max()),
    10 * (df[part]["angle"].size - 1) + 1,
)
axs[part]["main"].plot(
    x[part],
    cos_sq_fit(np.deg2rad(x[part]), *popt[part]),
    label="Fitted Curve",
    color="k",
    zorder=0.9,
)
axs[part]["main"].legend()
axs[part]["res"].scatter(
    df[part]["angle"],
    df[part]["residuals"],
    label="Residuals",
    color="r",
)
fig[part].tight_layout()

## Part 3: Two Polarizers

Insert a second polarizer between the first polarizer and the sensor.
Set the first polarizer to zero degrees and vary the angle of the second polarizer.
Record the sensor value as a function of the second polarizer angle from 0&deg; to 350&deg; in 10&deg; steps.

Graph your results.

In [ ]:
part: int = 3
ws[part] = gs.worksheet(f"Part {part}: Data")
df[part] = pd.DataFrame(
    data=np.asarray(
        ws[part].batch_get(
            ["A2:B37"],
            value_render_option=gspread.utils.ValueRenderOption.unformatted,
        )[0]
    ),
    columns=[
        "angle",
        "intensity",
    ],
)

intensity[part] = {
    "nominal": ufloat(
        df[part]["intensity"].mean(), df[part]["intensity"].std()
    ),
    "max": df[part]["intensity"].max(),
    "min": df[part]["intensity"].min(),
}
display(
    Markdown(
        Rf"Nominal Intensity: ${intensity[part]['nominal']:SL}\,\text{{lx}}$"
        "\n\n"
        Rf"Maximum Intensity: ${intensity[part]['max']:.1f}\,\text{{lx}}$"
        "\n\n"
        Rf"Minimum Intensity: ${intensity[part]['min']:.1f}\,\text{{lx}}$"
    )
)

popt[part], pcov[part] = opt.curve_fit(
    cos_sq_fit,
    np.deg2rad(df[part]["angle"]),
    df[part]["intensity"],
    p0=[
        df[part]["intensity"].max() - df[part]["intensity"].min(),
        0,
        df[part]["intensity"].min(),
    ],
)
perr[part] = np.sqrt(np.diag(pcov[part]))
height[part] = ufloat(popt[part][0], perr[part][0])
angular_offset[part] = ufloat(
    np.rad2deg(popt[part][1]), np.rad2deg(perr[part][1])
)
background_offset[part] = ufloat(popt[part][2], perr[part][2])

df[part]["residuals"] = df[part]["intensity"] - cos_sq_fit(
    np.deg2rad(df[part]["angle"]), *popt[part]
)

display(
    Markdown(
        R"Fit: $I\left(\theta\right) = "
        R"I_0 \cos^2\left(\theta + \theta_0\right) + I_\text{bg}$"
        "\n"
        Rf"- Height: $I_0 = {height[part]:SL}\,\text{{lx}}$"
        "\n"
        Rf"- Angular Offset: $\theta_0 = {angular_offset[part]:SL}\degree$"
        "\n"
        R"- Background Offset: $I_\text{{bg}} = "
        Rf"{background_offset[part]:SL}\,\text{{lx}}$"
        "\n\n"
        Rf"$I_0 / I_\text{{bg}} = {height[part] / background_offset[part]:SL}$"
    )
)

fig[part], axs[part] = plt.subplot_mosaic(
    [["main"], ["res"]],
    sharex=True,
    height_ratios=[1, 0.5],
)
axs[part]["main"].set_title(R"Transmitted Intensity vs. Polarizer Angle")
axs[part]["res"].set_xlabel(R"Polarizer Angle $\left(\degree\right)$")
axs[part]["main"].set_ylabel(R"Transmitted Intensity $\left(\text{lx}\right)$")
axs[part]["res"].set_ylabel(R"Residuals $\left(\text{lx}\right)$")
axs[part]["main"].scatter(
    df[part]["angle"],
    df[part]["intensity"],
    label="Measured Intensity",
)
x[part] = np.linspace(
    min(0, df[part]["angle"].min()),
    max(360, df[part]["angle"].max()),
    10 * (df[part]["angle"].size - 1) + 1,
)
axs[part]["main"].plot(
    x[part],
    cos_sq_fit(np.deg2rad(x[part]), *popt[part]),
    label="Fitted Curve",
    color="k",
    zorder=0.9,
)
axs[part]["main"].legend()
axs[part]["res"].scatter(
    df[part]["angle"],
    df[part]["residuals"],
    label="Residuals",
    color="r",
)
fig[part].tight_layout()

## Part 4: Three Polarizers

With the first polarizer still set to 0&deg;, set the second polarizer to 90&deg;.
Insert a third polarizer between the them.
Vary the angle of the middle polarizer and record the sensor reeding as a function of said angle (from 0&deg; to 350&deg; in 10&deg; steps).

What are the maximum and minimum values?

Is this what you expected?

How do you explain the results?

Graph your results.

In [ ]:
part: int = 4
ws[part] = gs.worksheet(f"Part {part}: Data")
df[part] = pd.DataFrame(
    data=np.asarray(
        ws[part].batch_get(
            ["A2:B37"],
            value_render_option=gspread.utils.ValueRenderOption.unformatted,
        )[0]
    ),
    columns=[
        "angle",
        "intensity",
    ],
)

intensity[part] = {
    "nominal": ufloat(
        df[part]["intensity"].mean(), df[part]["intensity"].std()
    ),
    "max": df[part]["intensity"].max(),
    "min": df[part]["intensity"].min(),
}
display(
    Markdown(
        Rf"Nominal Intensity: ${intensity[part]['nominal']:SL}\,\text{{lx}}$"
        "\n\n"
        Rf"Maximum Intensity: ${intensity[part]['max']:.1f}\,\text{{lx}}$"
        "\n\n"
        Rf"Minimum Intensity: ${intensity[part]['min']:.1f}\,\text{{lx}}$"
    )
)

popt[part], pcov[part] = opt.curve_fit(
    cos_sq_sin_sq_fit,
    np.deg2rad(df[part]["angle"]),
    df[part]["intensity"],
    p0=[
        df[part]["intensity"].max() - df[part]["intensity"].min(),
        0,
        df[part]["intensity"].min(),
    ],
)
perr[part] = np.sqrt(np.diag(pcov[part]))
height[part] = ufloat(popt[part][0], perr[part][0])
angular_offset[part] = ufloat(
    np.rad2deg(popt[part][1]), np.rad2deg(perr[part][1])
)
background_offset[part] = ufloat(popt[part][2], perr[part][2])

df[part]["residuals"] = df[part]["intensity"] - cos_sq_sin_sq_fit(
    np.deg2rad(df[part]["angle"]), *popt[part]
)

display(
    Markdown(
        R"Fit: $I\left(\theta\right) = "
        R"I_0 \cos^2\left(\theta + \theta_0\right) "
        R"\sin^2\left(\theta + \theta_0\right) + I_\text{bg}$"
        "\n"
        Rf"- Height: $I_0 = {height[part]:SL}\,\text{{lx}}$"
        "\n"
        Rf"- Angular Offset: $\theta_0 = {angular_offset[part]:SL}\degree$"
        "\n"
        R"- Background Offset: $I_\text{{bg}} = "
        Rf"{background_offset[part]:SL}\,\text{{lx}}$"
        "\n\n"
        Rf"$I_0 / I_\text{{bg}} = {height[part] / background_offset[part]:SL}$"
    )
)

fig[part], axs[part] = plt.subplot_mosaic(
    [["main"], ["res"]],
    sharex=True,
    height_ratios=[1, 0.5],
)
axs[part]["main"].set_title(R"Transmitted Intensity vs. Polarizer Angle")
axs[part]["res"].set_xlabel(R"Polarizer Angle $\left(\degree\right)$")
axs[part]["main"].set_ylabel(R"Transmitted Intensity $\left(\text{lx}\right)$")
axs[part]["res"].set_ylabel(R"Residuals $\left(\text{lx}\right)$")
axs[part]["main"].scatter(
    df[part]["angle"],
    df[part]["intensity"],
    label="Measured Intensity",
)
x[part] = np.linspace(
    min(0, df[part]["angle"].min()),
    max(360, df[part]["angle"].max()),
    10 * (df[part]["angle"].size - 1) + 1,
)
axs[part]["main"].plot(
    x[part],
    cos_sq_sin_sq_fit(np.deg2rad(x[part]), *popt[part]),
    label="Fitted Curve",
    color="k",
    zorder=0.9,
)
axs[part]["main"].legend()
axs[part]["res"].scatter(
    df[part]["angle"],
    df[part]["residuals"],
    label="Residuals",
    color="r",
)
fig[part].tight_layout()

## Part 5: Quarter Wave Plate

Replace the middle polarizer with a quarter-wave plate.
Measure the sensor value as a function of the waveplate angle from 0&deg; to 350&deg; in 10&deg; steps.

Graph your results.

Based on your measurements, what does the wave plate do the the polarization of the light?

In [ ]:
part: int = 5
ws[part] = gs.worksheet(f"Part {part}: Data")
df[part] = pd.DataFrame(
    data=np.asarray(
        ws[part].batch_get(
            ["A2:B37"],
            value_render_option=gspread.utils.ValueRenderOption.unformatted,
        )[0]
    ),
    columns=[
        "angle",
        "intensity",
    ],
)

intensity[part] = {
    "nominal": ufloat(
        df[part]["intensity"].mean(), df[part]["intensity"].std()
    ),
    "max": df[part]["intensity"].max(),
    "min": df[part]["intensity"].min(),
}
display(
    Markdown(
        Rf"Nominal Intensity: ${intensity[part]['nominal']:SL}\,\text{{lx}}$"
        "\n\n"
        Rf"Maximum Intensity: ${intensity[part]['max']:.1f}\,\text{{lx}}$"
        "\n\n"
        Rf"Minimum Intensity: ${intensity[part]['min']:.1f}\,\text{{lx}}$"
    )
)

popt[part], pcov[part] = opt.curve_fit(
    cos_sq_sin_sq_fit,
    np.deg2rad(df[part]["angle"]),
    df[part]["intensity"],
    p0=[
        df[part]["intensity"].max() - df[part]["intensity"].min(),
        0,
        df[part]["intensity"].min(),
    ],
)
perr[part] = np.sqrt(np.diag(pcov[part]))
height[part] = ufloat(popt[part][0], perr[part][0])
angular_offset[part] = ufloat(
    np.rad2deg(popt[part][1]), np.rad2deg(perr[part][1])
)
background_offset[part] = ufloat(popt[part][2], perr[part][2])

df[part]["residuals"] = df[part]["intensity"] - cos_sq_sin_sq_fit(
    np.deg2rad(df[part]["angle"]), *popt[part]
)

display(
    Markdown(
        R"Fit: $I\left(\theta\right) = "
        R"I_0 \cos^2\left(\theta + \theta_0\right) "
        R"\sin^2\left(\theta + \theta_0\right) + I_\text{bg}$"
        "\n"
        Rf"- Height: $I_0 = {height[part]:SL}\,\text{{lx}}$"
        "\n"
        Rf"- Angular Offset: $\theta_0 = {angular_offset[part]:SL}\degree$"
        "\n"
        R"- Background Offset: $I_\text{{bg}} = "
        Rf"{background_offset[part]:SL}\,\text{{lx}}$"
        "\n\n"
        Rf"$I_0 / I_\text{{bg}} = {height[part] / background_offset[part]:SL}$"
    )
)

fig[part], axs[part] = plt.subplot_mosaic(
    [["main"], ["res"]],
    sharex=True,
    height_ratios=[1, 0.5],
)
axs[part]["main"].set_title(R"Transmitted Intensity vs. Polarizer Angle")
axs[part]["res"].set_xlabel(R"Polarizer Angle $\left(\degree\right)$")
axs[part]["main"].set_ylabel(R"Transmitted Intensity $\left(\text{lx}\right)$")
axs[part]["res"].set_ylabel(R"Residuals $\left(\text{lx}\right)$")
axs[part]["main"].scatter(
    df[part]["angle"],
    df[part]["intensity"],
    label="Measured Intensity",
)
x[part] = np.linspace(
    min(0, df[part]["angle"].min()),
    max(360, df[part]["angle"].max()),
    10 * (df[part]["angle"].size - 1) + 1,
)
axs[part]["main"].plot(
    x[part],
    cos_sq_sin_sq_fit(np.deg2rad(x[part]), *popt[part]),
    label="Fitted Curve",
    color="k",
    zorder=0.9,
)
axs[part]["main"].legend()
axs[part]["res"].scatter(
    df[part]["angle"],
    df[part]["residuals"],
    label="Residuals",
    color="r",
)
fig[part].tight_layout()

## Lab Report

You will be doing your second lab report on this lab.

For your lab report, graph and fit the results for

- the two polarizers rotated relative to one another (part 3),
- the three polarizer configuration (part 4), and
- the two fixed polarizers with the waveplate in between (part 5).

Based on the fits, do the elements behave as you expect them to?
If there are discrepancies between your data and fits, can these be explained by systematic issues with the measurements?